# Complex of Rnase with DNA: deuteration and matching



#### Prerequisite 

In [ ]:
%matplotlib widget

import jscatter as js
import numpy as np
js.usempl(True)   # force matplotlib, not needed in Linux
import nglview as nv

#### load a protein DNA complex

In [ ]:
universe = js.bio.scatteringUniverse('2ez6',addHydrogen='pymol')
print(universe.select_atoms('type H').n_atoms, ' hydrogens')


#### look at the loaded structure

In [ ]:
w = nv.show_mdanalysis(universe)
w

In [ ]:
protein = universe.select_atoms('protein')
dna = universe.select_atoms('nucleic')

In [ ]:
v = nv.show_mdanalysis(protein)
v

### Set universe parameters for scattering and calc scattering for the parts

In [ ]:
universe.qlist = np.r_[0.1, 0.1:4:0.03]
universe.setSolvent(['1D2O1', '0H2O1'])

Icomplex = js.bio.nscatIntUniv(universe.atoms)
Iprotein = js.bio.nscatIntUniv(protein.atoms)
Idna = js.bio.nscatIntUniv(dna.atoms)

In [ ]:
p = js.mplot()
p.Xaxis(scale='log')
p.Yaxis(scale='log')
p.Plot(Icomplex,le='complex')
p.Plot(Iprotein,le='protein')
p.Plot(Idna,le='DNA')
p.Legend()


### Deuterate protein or DNA

In [ ]:
universe.atoms.deuteration = 0
# deuterate dna
dna.atoms.deuteration = 1
I_ddna = js.bio.nscatIntUniv(universe.atoms)
p.Plot(I_ddna,le='d-dna');

universe.atoms.deuteration = 0
# deuterate dna
protein.atoms.deuteration = 1
I_dprotein = js.bio.nscatIntUniv(universe.atoms)
p.Plot(I_dprotein,le='d-protein');
p.Legend()

### A matching experiment

In [ ]:
universe.atoms.deuteration = 0
universe.qlist = np.r_[0.1:4:0.1]
match = js.dL()

for d2o in np.r_[0:1.1:0.1]:
    universe.setSolvent([f'{d2o:.2f}D2O1', f'{1-d2o:.2f}H2O1'])
    I = js.bio.nscatIntUniv(protein,refreshVolume=False)
    match.append(I)


In [ ]:
p1 = js.mplot()
p1.Xaxis(label='$Q / nm^-1$',scale='log')
p1.Yaxis(label='I(Q) / a.u.',scale='log')
p1.Plot(match,le='$d2oFract')

p1.Legend()

In [ ]:
p2 = js.mplot()
p2.Xaxis(label='d2o content')
p2.Yaxis(label='I(0)',scale='log')
p2.plot(match.d2oFract,match.I0)
